In [1]:
import os

In [2]:
import re

In [37]:
import zipfile

In [4]:
import numpy as np

In [5]:
import pandas as pd

In [6]:
import xarray as xr

In [7]:
import mkgu

In [8]:
from mkgu.knownfile import KnownFile as kf

In [9]:
from mkgu.knownfile import FileRecord, Sighting

In [10]:
from mkgu.lookup import pwdb

In [25]:
from mkgu.assemblies import AssemblyModel, AssemblyStoreMap, AssemblyStoreModel

In [27]:
from mkgu.stimuli import ImageModel, AttributeModel, ImageMetaModel, StimulusSetModel, ImageStoreModel, \
    StimulusSetImageMap, ImageStoreMap

## Make the DataArray/.nc

In [12]:
gd = xr.open_dataarray("/Users/jjpr/dev/scratch/gallant_data/gallant-V1/data.nc")
gd

<xarray.DataArray (image_file_name: 189305, neuroid: 23)>
[4354015 values with dtype=float64]
Coordinates:
  * image_file_name  (image_file_name) object 'gallant\\V1Data\\NatRev/stimuli/r0148A/NatRev.MountLake.40.150.imsm/1.jpg' ...
  * neuroid          (neuroid) object 'r0148A' 'r0150B' 'r0154B' 'r0156A' ...

In [13]:
gd = gd.T.rename({"image_file_name": "presentation"})

In [14]:
gd.coords["presentation_id"] = ("presentation", range(gd.shape[1]))

In [15]:
gd.coords["neuroid_id"] = ("neuroid", gd["neuroid"].values)

In [16]:
file_base = "/Users/jjpr/dev/scratch/gallant_data/gallant-V1/stimuli"
def massage_file_name(file_name):
    split = re.split("\\\\|/", file_name)
    relative_path = os.path.join(*split[4:])
    full_path = os.path.join(file_base, relative_path)
    basename = split[-1]
    exists = os.path.exists(full_path)
    sha1 = kf(full_path).sha1
    result = {
        "image_file_path_original": relative_path,
        "image_id": sha1
    }
    return result

In [17]:
df_massage = pd.DataFrame(list(map(massage_file_name, gd["presentation"].values)))
df_massage

,image_file_path_original,image_id
0,r0148A/NatRev.MountLake.40.150.imsm/1.jpg,33613a99bdc22fe498864bf32b798e018a961ed7
1,r0148A/NatRev.MountLake.40.150.imsm/2.jpg,8bfba1c6476cab79b0b9d8a304a5a9cd67139b7c
2,r0148A/NatRev.MountLake.40.150.imsm/3.jpg,7a8e6c239c4e0781bccfb504eba686451e2c2aa7
3,r0148A/NatRev.MountLake.40.150.imsm/4.jpg,5cf9e8b084d15a014ea185060ea107026dae7fe0
4,r0148A/NatRev.MountLake.40.150.imsm/5.jpg,5496b7a29ceb2610fa24b09af95d6483672ec922
5,r0148A/NatRev.MountLake.40.150.imsm/6.jpg,92db51917cfa571416896b6f761977f388bfcecd
6,r0148A/NatRev.MountLake.40.150.imsm/7.jpg,0604dd18a078cb7c5776159b3407de9e79808f7a
7,r0148A/NatRev.MountLake.40.150.imsm/8.jpg,a9c35e09b71a32ddb76ac3eda02cf7f9d1afba37
8,r0148A/NatRev.MountLake.40.150.imsm/9.jpg,9ab1e98e3deeda077f0624fd02d7fd0b0ee67124
9,r0148A/NatRev.MountLake.40.150.imsm/10.jpg,6cb3acba902afb879c4091ec3e5feca193a489a3


In [18]:
for column in df_massage.columns:
    gd.coords[column] = ("presentation", df_massage[column])
gd

<xarray.DataArray (neuroid: 23, presentation: 189305)>
array([[  0.,   0.,   0., ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       ..., 
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,   0.,   0.,   0.]])
Coordinates:
  * presentation              (presentation) object 'gallant\\V1Data\\NatRev/stimuli/r0148A/NatRev.MountLake.40.150.imsm/1.jpg' ...
  * neuroid                   (neuroid) object 'r0148A' 'r0150B' 'r0154B' ...
    presentation_id           (presentation) int64 0 1 2 3 4 5 6 7 8 9 10 11 ...
    neuroid_id                (neuroid) object 'r0148A' 'r0150B' 'r0154B' ...
    image_file_path_original  (presentation) object 'r0148A/NatRev.MountLake.40.150.imsm/1.jpg' ...
    image_id                  (presentation) object '33613a99bdc22fe498864bf32b798e018a961ed7' ...

In [19]:
gd.reset_index(["neuroid", "presentation"], drop=True, inplace=True)
gd

<xarray.DataArray (neuroid: 23, presentation: 189305)>
array([[  0.,   0.,   0., ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       ..., 
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,   0.,   0.,   0.]])
Coordinates:
    presentation_id           (presentation) int64 0 1 2 3 4 5 6 7 8 9 10 11 ...
    neuroid_id                (neuroid) object 'r0148A' 'r0150B' 'r0154B' ...
    image_file_path_original  (presentation) object 'r0148A/NatRev.MountLake.40.150.imsm/1.jpg' ...
    image_id                  (presentation) object '33613a99bdc22fe498864bf32b798e018a961ed7' ...
Dimensions without coordinates: neuroid, presentation

In [20]:
gd.to_netcdf("gallant_v1_single_electrode.nc")

## Make the image zip file

In [29]:
df_image_meta = pd.DataFrame({"image_id": np.unique(gd["image_id"].values)})

In [30]:
def first_dupe(sha1):
    fr = FileRecord.get(sha1=sha1)
    return fr.sightings[0].location
# order is not guaranteed, so on subsequent runs test that you got the same result, see below

In [31]:
df_image_meta["first_dupe"] = list(map(first_dupe, df_image_meta["image_id"]))
df_image_meta

,image_id,first_dupe
0,000035942277141353c41e43868fed37718f3264,/Users/jjpr/dev/scratch/gallant_data/gallant-V...
1,0000365716935db4f6c4e8fe01784c0f6986f5a3,/Users/jjpr/dev/scratch/gallant_data/gallant-V...
2,00003a9921d3b3db0e76898191289c7fe8150975,/Users/jjpr/dev/scratch/gallant_data/gallant-V...
3,0000a8aa19d0a68f278f240b716d5bbc8f0c6691,/Users/jjpr/dev/scratch/gallant_data/gallant-V...
4,00010bb8a48e7a0997cb31c21662d826603ee4a8,/Users/jjpr/dev/scratch/gallant_data/gallant-V...
5,0001df478512ce4b526bb9f3649bd604d417ace2,/Users/jjpr/dev/scratch/gallant_data/gallant-V...
6,00030a3bd87f598ce692026432462f552a545520,/Users/jjpr/dev/scratch/gallant_data/gallant-V...
7,000539371e6d73f77563dc34e58cb6d361bebf00,/Users/jjpr/dev/scratch/gallant_data/gallant-V...
8,00053e951981d431da7e75ac529e627adbceb054,/Users/jjpr/dev/scratch/gallant_data/gallant-V...
9,00058e96f1df6e5249a5f7ee6e9705e64a25c9c4,/Users/jjpr/dev/scratch/gallant_data/gallant-V...


In [51]:
def get_relative(path, base):
    split_path = path.split("/")
    split_base = base.split("/")
    target_path = "/".join(split_path[len(split_base):])
    return target_path

In [52]:
df_image_meta["relative_path"] = list(map(lambda x: get_relative(x, file_base), df_image_meta["first_dupe"]))
df_image_meta

,image_id,first_dupe,copied,relative_path
0,000035942277141353c41e43868fed37718f3264,/Users/jjpr/dev/scratch/gallant_data/gallant-V...,True,r0210A/test.natrev.24_Hz.1.imsm/3795.jpg
1,0000365716935db4f6c4e8fe01784c0f6986f5a3,/Users/jjpr/dev/scratch/gallant_data/gallant-V...,True,r0170A/test.natrev.size.imsm/1182.jpg
2,00003a9921d3b3db0e76898191289c7fe8150975,/Users/jjpr/dev/scratch/gallant_data/gallant-V...,True,r0148A/NatRev.MountLake.40.150.imsm/4229.jpg
3,0000a8aa19d0a68f278f240b716d5bbc8f0c6691,/Users/jjpr/dev/scratch/gallant_data/gallant-V...,True,r0210A/test.natrev.1.imsm/2280.jpg
4,00010bb8a48e7a0997cb31c21662d826603ee4a8,/Users/jjpr/dev/scratch/gallant_data/gallant-V...,True,r0168B/test.natrev.clown.24sm.imsm/5085.jpg
5,0001df478512ce4b526bb9f3649bd604d417ace2,/Users/jjpr/dev/scratch/gallant_data/gallant-V...,True,r0217B/test.natrev.forestferns.20_pix.imsm/485...
6,00030a3bd87f598ce692026432462f552a545520,/Users/jjpr/dev/scratch/gallant_data/gallant-V...,True,r0210A/test.natrev.24_Hz.1.imsm/5517.jpg
7,000539371e6d73f77563dc34e58cb6d361bebf00,/Users/jjpr/dev/scratch/gallant_data/gallant-V...,True,r0169B/test.natrev.mountlake.imsm/1028.jpg
8,00053e951981d431da7e75ac529e627adbceb054,/Users/jjpr/dev/scratch/gallant_data/gallant-V...,True,r0158A/test.natrev.mountlake.20sm.imsm/6297.jpg
9,00058e96f1df6e5249a5f7ee6e9705e64a25c9c4,/Users/jjpr/dev/scratch/gallant_data/gallant-V...,True,r0217B/test.natrev.andros.20_pix.imsm/2320.jpg


In [40]:
target_zip_path = "/Users/jjpr/.mkgu/data/gallant.David2004/gallant_crcns_v1_stimuli.zip"

In [53]:
with zipfile.ZipFile(target_zip_path, 'w') as target_zip:
    for image in df_image_meta.itertuples():
        target_zip.write(image.first_dupe, arcname=image.relative_path)

In [54]:
containing_dir = os.path.dirname(target_zip_path)
with zipfile.ZipFile(target_zip_path, 'r') as new_zip:
    new_zip.extractall(containing_dir)

In [55]:
def copied(source):
    split = source.split("/")
    target = os.path.join(containing_dir, "/".join(split[8:]))
    return os.path.exists(target)

In [56]:
df_image_meta["copied"] = list(map(copied, df_image_meta["first_dupe"]))
df_image_meta

,image_id,first_dupe,copied,relative_path
0,000035942277141353c41e43868fed37718f3264,/Users/jjpr/dev/scratch/gallant_data/gallant-V...,True,r0210A/test.natrev.24_Hz.1.imsm/3795.jpg
1,0000365716935db4f6c4e8fe01784c0f6986f5a3,/Users/jjpr/dev/scratch/gallant_data/gallant-V...,True,r0170A/test.natrev.size.imsm/1182.jpg
2,00003a9921d3b3db0e76898191289c7fe8150975,/Users/jjpr/dev/scratch/gallant_data/gallant-V...,True,r0148A/NatRev.MountLake.40.150.imsm/4229.jpg
3,0000a8aa19d0a68f278f240b716d5bbc8f0c6691,/Users/jjpr/dev/scratch/gallant_data/gallant-V...,True,r0210A/test.natrev.1.imsm/2280.jpg
4,00010bb8a48e7a0997cb31c21662d826603ee4a8,/Users/jjpr/dev/scratch/gallant_data/gallant-V...,True,r0168B/test.natrev.clown.24sm.imsm/5085.jpg
5,0001df478512ce4b526bb9f3649bd604d417ace2,/Users/jjpr/dev/scratch/gallant_data/gallant-V...,True,r0217B/test.natrev.forestferns.20_pix.imsm/485...
6,00030a3bd87f598ce692026432462f552a545520,/Users/jjpr/dev/scratch/gallant_data/gallant-V...,True,r0210A/test.natrev.24_Hz.1.imsm/5517.jpg
7,000539371e6d73f77563dc34e58cb6d361bebf00,/Users/jjpr/dev/scratch/gallant_data/gallant-V...,True,r0169B/test.natrev.mountlake.imsm/1028.jpg
8,00053e951981d431da7e75ac529e627adbceb054,/Users/jjpr/dev/scratch/gallant_data/gallant-V...,True,r0158A/test.natrev.mountlake.20sm.imsm/6297.jpg
9,00058e96f1df6e5249a5f7ee6e9705e64a25c9c4,/Users/jjpr/dev/scratch/gallant_data/gallant-V...,True,r0217B/test.natrev.andros.20_pix.imsm/2320.jpg


In [57]:
all(df_image_meta["copied"])

True

In [58]:
ls $target_base

r0148A/ r0156A/ r0162B/ r0168B/ r0170A/ r0208D/ r0211A/ r0215B/
r0154B/ r0158A/ r0164C/ r0169B/ r0206B/ r0210A/ r0212B/ r0217B/


## Make the StimulusSet lookup meta

In [23]:
pwdb.connect(reuse_if_open=True)

True

In [28]:
pwdb.create_tables(models=[ImageModel, AttributeModel, ImageMetaModel, StimulusSetModel, ImageStoreModel, StimulusSetImageMap, ImageStoreMap])

In [59]:
gallant_v1_images, created = StimulusSetModel.get_or_create(name="David2004")

In [60]:
gallant_v1_image_store, created = ImageStoreModel.get_or_create(location_type="S3", store_type="zip",
                                  location="https://mkgu-gallant-crcns.s3.amazonaws.com/gallant_crcns_v1_stimuli.zip")

In [61]:
eav_image_file_sha1 = AttributeModel(name="image_file_sha1", type="str")
eav_image_file_path_unique = AttributeModel(name="image_file_path_unique", type="str")

eav_image_file_sha1.save()
eav_image_file_path_unique.save()

1

In [62]:
for image in df_image_meta.itertuples():
    pw_image = ImageModel(image_id=image.image_id)
    pw_stimulus_set_image_map = StimulusSetImageMap(stimulus_set=gallant_v1_images, image=pw_image)
    pw_image_image_store_map = ImageStoreMap(image=pw_image, image_store=gallant_v1_image_store,
                                             path=image.relative_path)
    pw_image.save()
    pw_stimulus_set_image_map.save()
    pw_image_image_store_map.save()
    ImageMetaModel(image=pw_image, attribute=eav_image_file_sha1, value=str(image.image_id)).save()
    ImageMetaModel(image=pw_image, attribute=eav_image_file_path_unique, value=str(image.relative_path)).save()

In [63]:
gallant_v1_stimulus_set = mkgu.get_stimulus_set("David2004")
gallant_v1_stimulus_set

,id,image_id,image_file_sha1,image_file_path_unique
0,5761,000035942277141353c41e43868fed37718f3264,000035942277141353c41e43868fed37718f3264,r0210A/test.natrev.24_Hz.1.imsm/3795.jpg
1,5762,0000365716935db4f6c4e8fe01784c0f6986f5a3,0000365716935db4f6c4e8fe01784c0f6986f5a3,r0170A/test.natrev.size.imsm/1182.jpg
2,5763,00003a9921d3b3db0e76898191289c7fe8150975,00003a9921d3b3db0e76898191289c7fe8150975,r0148A/NatRev.MountLake.40.150.imsm/4229.jpg
3,5764,0000a8aa19d0a68f278f240b716d5bbc8f0c6691,0000a8aa19d0a68f278f240b716d5bbc8f0c6691,r0210A/test.natrev.1.imsm/2280.jpg
4,5765,00010bb8a48e7a0997cb31c21662d826603ee4a8,00010bb8a48e7a0997cb31c21662d826603ee4a8,r0168B/test.natrev.clown.24sm.imsm/5085.jpg
5,5766,0001df478512ce4b526bb9f3649bd604d417ace2,0001df478512ce4b526bb9f3649bd604d417ace2,r0217B/test.natrev.forestferns.20_pix.imsm/485...
6,5767,00030a3bd87f598ce692026432462f552a545520,00030a3bd87f598ce692026432462f552a545520,r0210A/test.natrev.24_Hz.1.imsm/5517.jpg
7,5768,000539371e6d73f77563dc34e58cb6d361bebf00,000539371e6d73f77563dc34e58cb6d361bebf00,r0169B/test.natrev.mountlake.imsm/1028.jpg
8,5769,00053e951981d431da7e75ac529e627adbceb054,00053e951981d431da7e75ac529e627adbceb054,r0158A/test.natrev.mountlake.20sm.imsm/6297.jpg
9,5770,00058e96f1df6e5249a5f7ee6e9705e64a25c9c4,00058e96f1df6e5249a5f7ee6e9705e64a25c9c4,r0217B/test.natrev.andros.20_pix.imsm/2320.jpg


## Make the DataAssembly lookup meta

In [64]:
pwdb.create_tables(models=[AssemblyModel, AssemblyStoreMap, AssemblyStoreModel])

In [65]:
assy = AssemblyModel(name="gallant.David2004", assembly_class="NeuronRecordingAssembly",
                     stimulus_set=gallant_v1_images)
assy.save()

1

In [66]:
store = AssemblyStoreModel(assembly_type="netCDF",
                           location_type="S3",
                           location="https://mkgu-gallant-crcns.s3.amazonaws.com/gallant_v1_single_electrode.nc")
store.save()

1

In [67]:
assy_store_map = AssemblyStoreMap(assembly_model=assy, assembly_store_model=store, role="gallant.David2004")
assy_store_map.save()

1

In [68]:
gallant_v1 = mkgu.get_assembly("gallant.David2004")
gallant_v1

<xarray.NeuronRecordingAssembly (neuroid: 23, presentation: 189305)>
array([[  0.,   0.,   0., ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       ..., 
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,   0.,   0.,   0.]])
Coordinates:
  * neuroid                   (neuroid) object 'r0148A' 'r0150B' 'r0154B' ...
  * presentation              (presentation) MultiIndex
  - presentation_id           (presentation) int64 0 1 2 3 4 5 6 7 8 9 10 11 ...
  - image_file_path_original  (presentation) object 'r0148A/NatRev.MountLake.40.150.imsm/1.jpg' ...
  - image_id                  (presentation) object '33613a99bdc22fe498864bf32b798e018a961ed7' ...
  - id                        (presentation) int64 29882 71211 63228 49398 ...
  - image_file_sha1           (presentation) object '33613a99bdc22fe498864bf32b798e018a961ed7' ...
  - image_fi

In [69]:
len(np.unique(gallant_v1["image_file_path_original"].values))

189305

In [70]:
len(np.unique(gallant_v1["image_file_path_unique"].values))

120017